In [ ]:
# Import necessary components from your module
from ipyleaflet import WidgetControl, ImageOverlay
from hat.mapping.visualisation import GeoJSONLayerManager, InteractiveMap, create_circle_markers, create_gradient_legend, make_line_click_handler, make_style_callback, vector_style, create_polygon_legend
import matplotlib.pyplot as plt
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
from io import BytesIO
import base64

# Define a colormap and normalization (if needed)
cmap = plt.cm.get_cmap('PRGn')
norm = plt.Normalize(vmin=-10, vmax=10)

gdf_station = gpd.read_file("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/station.geojson")

station_layer = create_circle_markers(gdf_station, 'near2new_distance_km', 0, 'blue', 'black', "Station")
nearest_grid_layer = GeoJSONLayerManager("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/nearest_grid.geojson", 
                                         style_callback=lambda feature: vector_style(feature, 'black', 0.5), 
                                         name="Near Grid")
new_grid_layer = GeoJSONLayerManager("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/new_grid.geojson", 
                                     style_callback=lambda feature: vector_style(feature, 'blue', 0.5), 
                                     name="New Grid")
line_new_layer = GeoJSONLayerManager("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/station2grid_new_line.geojson", 
                                     style_callback=make_style_callback('new_area_diff', cmap, norm), 
                                     name="New Area Difference")
river_overlay = GeoJSONLayerManager("riv_seq_europe.geojson", 
                                     style_callback=lambda feature: vector_style(feature, 'blue', 0.1), 
                                     name="River Grid")

In [ ]:
# Create an InteractiveMap instance
my_map = InteractiveMap()
# Add layers to the map
my_map.add_layer(river_overlay)
my_map.map.add_layer(station_layer)
my_map.add_layer(nearest_grid_layer)
my_map.add_layer(new_grid_layer)
my_map.add_layer(line_new_layer)

# Add line click handlers after the layer has been added to the map
if line_new_layer.layer:
    line_new_layer.layer.on_click(
        make_line_click_handler('station_name', 'station_area', 
                                'near_grid_area','new_grid_area',
                                'near_distance_km', 'new_distance_km', 
                                my_map.map))
                                
# Create the legend widget
legend_widget = create_gradient_legend(cmap, -10, 10)
colors = ["blue", "black"]  # Colors used in your polygon styles
labels = ["Moved", "Not Moved"]  # Labels for each color
polygon_legend = create_polygon_legend(colors, labels)


my_map.map.add_control(WidgetControl(widget=legend_widget, position='bottomright'))
my_map.map.add_control(WidgetControl(widget=polygon_legend, position='bottomleft'))

# Display the map using IPython's display function
display(my_map.map)

In [1]:
from hat.mapping.evaluation import calculate_mae, calculate_rmse, count_within_abs_error_range
import pandas as pd

df = pd.read_csv("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/station.csv")
me = calculate_mae(df, 'manual_area', 'new_grid_area')
rmse = calculate_rmse(df, 'manual_area', 'new_grid_area')
acceptable_count = count_within_abs_error_range(df, 'manual_area', 'new_grid_area', 1 , 10)

Mean Abs. Error (MAE) % between manual_area and new_grid_area: 337.3483295154547%
RMSE between manual_area and new_grid_area: 38608.96790225597 km2
Count of rows with absolute error % between manual_area and new_grid_area in the range of (1%, 10%): 450 / 1324


In [ ]:
import subprocess
import pandas as pd
import numpy as np
from hat.mapping.evaluation import calculate_rmse, count_within_range, update_config_file

config_file_path = "/home/dadiyorto/freelance/02_ort_ecmwf/dev/hat/hat/mapping/input.json"

# Define your parameter ranges
max_cells_range = range(1, 11)  # From 1 to 10
max_diff_range = range(2, 21, 2)  # From 2 to 20 with a step of 2

# Initialize matrices for RMSE and Acceptable Count
rmse_matrix = np.zeros((len(max_cells_range), len(max_diff_range)))
count_matrix = np.zeros((len(max_cells_range), len(max_diff_range)))

for i, max_cells in enumerate(max_cells_range):
    for j, max_diff in enumerate(max_diff_range):
        # Update the config file for each combination
        update_config_file(config_file_path, max_cells, max_diff)

        # Run the script
        subprocess.run(["/home/dadiyorto/freelance/02_ort_ecmwf/dev/hat/hat/mapping/station_mapping.py", config_file_path], check=True)

        # Load the updated CSV file
        df = pd.read_csv("~/freelance/02_ort_ecmwf/dev/hat/hat/mapping/station.csv")
        
        # Perform your evaluations
        rmse = calculate_rmse(df, 'new_grid_area', 'manual_area')  # Set print to False in function
        acceptable_count = count_within_range(df, 'new_area_diff', 1, 10)  # Set print to False in function

        # Store results in the matrices
        rmse_matrix[i, j] = rmse
        count_matrix[i, j] = acceptable_count

# Convert matrices to DataFrame for better readability
rmse_df = pd.DataFrame(rmse_matrix, index=max_cells_range, columns=max_diff_range)
count_df = pd.DataFrame(count_matrix, index=max_cells_range, columns=max_diff_range)

